In [2]:
# cmaes_ellipsoid.py
import numpy as np
from dataclasses import dataclass

# ----------------------------
#  Objective functions
# ----------------------------
def ellipsoid(x: np.ndarray, alpha: float = 1e6) -> float:
    """
    Separable Ellipsoid:
    f(x) = Σ alpha^{(i-1)/(n-1)} * x_i^2  (d[0]=1, d[-1]=alpha)
    Minimum global di x=0 dengan f=0.
    """
    x = np.asarray(x)
    n = x.size
    if n == 1:
        d = np.array([1.0])
    else:
        d = alpha ** np.linspace(0.0, 1.0, n)
    return float(np.sum(d * x * x))

def make_rotation_matrix(n: int, seed: int | None = 42) -> np.ndarray:
    """
    Matriks rotasi ortonormal Q via QR (determinant +1).
    """
    rng = np.random.default_rng(seed)
    Q, _ = np.linalg.qr(rng.normal(size=(n, n)))
    if np.linalg.det(Q) < 0:
        Q[:, 0] = -Q[:, 0]
    return Q

def rotated_ellipsoid(x: np.ndarray, Q: np.ndarray, alpha: float = 1e6) -> float:
    """
    Rotated (non-separable) Ellipsoid:
    f(x) = Σ alpha^{(i-1)/(n-1)} * (Qx)_i^2
    """
    x = np.asarray(x)
    n = x.size
    z = Q @ x
    if n == 1:
        d = np.array([1.0])
    else:
        d = alpha ** np.linspace(0.0, 1.0, n)
    return float(np.sum(d * z * z))

# ----------------------------
#  CMA-ES core
# ----------------------------
@dataclass
class CMAESConfig:
    x0: np.ndarray          # initial mean
    sigma0: float           # initial global step-size
    lambda_: int = None     # offspring size; default: 4 + floor(3*ln(n))
    seed: int | None = 123
    diag_only: bool = False # if True, sep-CMA (diagonal C) for high-dim
    max_iter: int = 2000
    target_f: float = 1e-12
    tolx: float = 1e-12
    tolcond: float = 1e14   # condition number tolerance for C

class CMAES:
    def __init__(self, cfg: CMAESConfig):
        self.rng = np.random.default_rng(cfg.seed)
        self.xmean = np.asarray(cfg.x0, dtype=float)
        self.n = self.xmean.size
        self.sigma = float(cfg.sigma0)
        self.lam = cfg.lambda_ or (4 + int(np.floor(3 * np.log(self.n))))
        # mu and recombination weights
        self.mu = self.lam // 2
        w = np.log(self.mu + 0.5) - np.log(np.arange(1, self.mu + 1))
        self.w = w / np.sum(w)
        self.mueff = (np.sum(self.w) ** 2) / np.sum(self.w**2)

        # Strategy parameters for C and sigma
        self.cc = (4 + self.mueff / self.n) / (self.n + 4 + 2 * self.mueff / self.n)
        self.cs = (self.mueff + 2) / (self.n + self.mueff + 5)
        self.c1 = 2 / ((self.n + 1.3)**2 + self.mueff)
        self.cmu = min(1 - self.c1, 2 * (self.mueff - 2 + 1 / self.mueff) / ((self.n + 2)**2 + self.mueff))
        self.damps = 1 + 2 * max(0, np.sqrt((self.mueff - 1) / (self.n + 1)) - 1) + self.cs

        # Dynamic state
        self.pc = np.zeros(self.n)
        self.ps = np.zeros(self.n)
        self.diag_only = cfg.diag_only
        if self.diag_only:
            self.Cd = np.ones(self.n)        # diagonal of C
            self.B = np.eye(self.n)          # not used, but keep for API symmetry
            self.D = np.ones(self.n)
        else:
            self.C = np.eye(self.n)
            self.B = np.eye(self.n)
            self.D = np.ones(self.n)
        self.inv_sqrt_C = np.eye(self.n)
        self.chiN = np.sqrt(self.n) * (1 - 1/(4*self.n) + 1/(21*self.n**2))  # E||N(0,I)||

        self.max_iter = cfg.max_iter
        self.target_f = cfg.target_f
        self.tolx = cfg.tolx
        self.tolcond = cfg.tolcond

    def _sample(self):
        if self.diag_only:
            # z ~ N(0, I), y = B*D*z = D*z (diagonal)
            z = self.rng.normal(size=(self.lam, self.n))
            y = z * self.D  # scale columns
        else:
            z = self.rng.normal(size=(self.lam, self.n))
            y = (self.B * self.D) @ z.T     # B*diag(D)*z^T
            y = y.T
        x = self.xmean + self.sigma * y
        return x, y, z

    def _update_eigensystem(self):
        if self.diag_only:
            # maintain diagonal only: C = diag(Cd), eig(B=I, D=sqrt(Cd))
            self.D = np.sqrt(np.maximum(self.Cd, 1e-30))
            self.inv_sqrt_C = np.diag(1.0 / self.D)
            return
        # Full eigendecomposition
        Csym = (self.C + self.C.T) * 0.5
        D2, B = np.linalg.eigh(Csym)
        D2 = np.maximum(D2, 1e-30)
        self.D = np.sqrt(D2)
        self.B = B
        self.inv_sqrt_C = (B @ np.diag(1.0 / self.D) @ B.T)

    def ask(self):
        x, y, z = self._sample()
        return x, y, z

    def tell(self, y, z, fitness):
        # sort by fitness
        idx = np.argsort(fitness)
        ysel = y[idx[:self.mu], :]
        zsel = z[idx[:self.mu], :]
        xold = self.xmean.copy()

        # recombination
        y_w = np.tensordot(self.w, ysel, axes=(0, 0))   # weighted mean in y-space
        self.xmean = xold + self.sigma * y_w

        # update evolution paths
        self.ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * (self.inv_sqrt_C @ (self.xmean - xold) / self.sigma)
        hsig = (np.linalg.norm(self.ps) / np.sqrt(1 - (1 - self.cs) ** (2)) / self.chiN) < (1.4 + 2 / (self.n + 1))
        self.pc = (1 - self.cc) * self.pc + hsig * np.sqrt(self.cc * (2 - self.cc) * self.mueff) * ((self.xmean - xold) / self.sigma)

        # covariance update
        if self.diag_only:
            # rank-one
            self.Cd = (1 - self.c1 - self.cmu) * self.Cd + self.c1 * (self.pc**2)
            # rank-mu using diagonal of weighted outer products
            cov_diag = np.sum(self.w[:, None] * (ysel**2), axis=0)
            self.Cd += self.cmu * cov_diag
        else:
            rank_one = np.outer(self.pc, self.pc)
            # rank-mu in y-space -> transform to C-space by outer
            Cmu = np.zeros_like(self.C)
            for wi, yi in zip(self.w, ysel):
                Cmu += wi * np.outer(yi, yi)
            self.C = (1 - self.c1 - self.cmu) * self.C + self.c1 * rank_one + self.cmu * Cmu

        # step-size update
        self.sigma *= np.exp((self.cs / self.damps) * (np.linalg.norm(self.ps) / self.chiN - 1))

        # occasional eigendecomposition
        if self.diag_only:
            # keep Cd in reasonable range
            self.Cd = np.clip(self.Cd, 1e-30, 1e30)
        else:
            # condition number guard (optional)
            if (np.max(self.D) / np.min(self.D)) > self.tolcond:
                # mild regularization
                eps = 1e-12
                self.C += eps * np.eye(self.n)
        # refresh eigensystem every ~ n/(c1+cmu)/10 steps or just every iteration (simple & safe)
        self._update_eigensystem()

    def stop(self, best_f, iter_):
        # stopping criteria
        if best_f <= self.target_f:
            return True, "target_f reached"
        if self.sigma * np.max(self.D) < self.tolx:
            return True, "tolx (step-size too small)"
        if iter_ >= self.max_iter:
            return True, "max_iter reached"
        return False, ""

# ----------------------------
#  Example run
# ----------------------------
def run_cmaes_on_ellipsoid(n=20, rotated=False, alpha=1e6, diag_only=False, seed=7):
    # initial mean and sigma
    x0 = np.ones(n) * 3.0
    sigma0 = 2.0

    cfg = CMAESConfig(
        x0=x0,
        sigma0=sigma0,
        lambda_=None,       # default
        seed=seed,
        diag_only=diag_only,
        max_iter=5000,
        target_f=1e-10,
        tolx=1e-12,
        tolcond=1e14
    )
    opt = CMAES(cfg)

    if rotated:
        Q = make_rotation_matrix(n, seed=seed + 100)
        f = lambda x: rotated_ellipsoid(x, Q, alpha=alpha)
    else:
        f = lambda x: ellipsoid(x, alpha=alpha)

    best_x, best_f = opt.xmean.copy(), f(opt.xmean)
    for it in range(1, opt.max_iter + 1):
        X, Y, Z = opt.ask()
        fit = np.array([f(xi) for xi in X])
        ib = np.argmin(fit)
        if fit[ib] < best_f:
            best_f = fit[ib]
            best_x = X[ib].copy()

        opt.tell(Y, Z, fit)
        done, reason = opt.stop(best_f, it)
        if it % 50 == 0 or done:
            print(f"[{it:5d}] fbest={best_f:.3e}  sigma={opt.sigma:.3e}")
        if done:
            print("Stop:", reason)
            break

    return best_x, best_f

if __name__ == "__main__":
    # Contoh 1: Ellipsoid separable, full covariance
    print("=== CMA-ES on Ellipsoid (separable, full-C) ===")
    bx, bf = run_cmaes_on_ellipsoid(n=20, rotated=False, alpha=1e6, diag_only=False, seed=11)
    print("Best f:", bf)

    # Contoh 2: Rotated Ellipsoid (non-separable), full covariance
    print("\n=== CMA-ES on Rotated Ellipsoid (full-C) ===")
    bx2, bf2 = run_cmaes_on_ellipsoid(n=20, rotated=True, alpha=1e6, diag_only=False, seed=11)
    print("Best f:", bf2)

    # Contoh 3: Ellipsoid dimensi tinggi dengan sep-CMA (diagonal C) — sangat cepat & ringan memori
    print("\n=== CMA-ES on Ellipsoid (sep-CMA, diagonal C) ===")
    bx3, bf3 = run_cmaes_on_ellipsoid(n=200, rotated=False, alpha=1e6, diag_only=True, seed=11)
    print("Best f:", bf3)


=== CMA-ES on Ellipsoid (separable, full-C) ===
[   50] fbest=4.105e+05  sigma=1.042e+00
[  100] fbest=6.244e+04  sigma=2.747e-01
[  150] fbest=4.327e+04  sigma=2.032e-01
[  200] fbest=2.520e+04  sigma=1.371e-01
[  250] fbest=1.783e+04  sigma=1.094e-01
[  300] fbest=1.212e+04  sigma=9.510e-02
[  350] fbest=9.309e+03  sigma=8.035e-02
[  400] fbest=7.303e+03  sigma=7.417e-02
[  450] fbest=5.864e+03  sigma=6.024e-02
[  500] fbest=4.669e+03  sigma=1.450e-01
[  550] fbest=3.834e+03  sigma=9.417e-02
[  600] fbest=3.020e+03  sigma=9.895e-02
[  650] fbest=2.535e+03  sigma=8.117e-02
[  700] fbest=2.030e+03  sigma=7.032e-02
[  750] fbest=1.595e+03  sigma=1.066e-01
[  800] fbest=1.307e+03  sigma=1.015e-01
[  850] fbest=1.053e+03  sigma=1.081e-01
[  900] fbest=8.848e+02  sigma=1.279e-01
[  950] fbest=6.468e+02  sigma=1.433e-01
[ 1000] fbest=4.932e+02  sigma=1.567e-01
[ 1050] fbest=3.351e+02  sigma=1.958e-01
[ 1100] fbest=2.389e+02  sigma=1.441e-01
[ 1150] fbest=1.641e+02  sigma=1.689e-01
[ 1200] f